# Feature engineering notebook

This is the final version of feature engineering notebook for solar model where we prepare our data for prediction. Prediction will be placed in a separate notebook. Going a little-bit upfront, we will be trying several regression models with our data set, including Random Forest algorithm and perhaps Neural Networks. Due to decision trees nature, the features will have to be engineered slightly differently:
> Example: 
> * Random forests do not benefit from too large feature spaces, since they select at random sqrt(p) or p/3 features, where p is total number of features. Therefore, when working with discretization and encoding of some of the numerical variables it is beneficial not to increase the feature space by one-hot encoding and using dummy variables. Instead, we will keep them in the same feature as a categorical variable.
> * For Neural Networks we would perform data scaling that is unneccessary for decision tree algorithms.

### Important notice: variable division transformations are done on the Train set

Checklist and a quick overview of feature engineering steps:

* missing values
* outliers
* discretization
* gaussian transformation (?)
* new features
* feature selection (+FMI API recieved data sneakpeak)

# Prepare the data  
Notes:    
- Work on copies of the data (keep the original dataset intact).  
- Write functions for all data transformations you apply, for five reasons:  
    - So you can easily prepare the data the next time you get a fresh dataset  
    - So you can apply these transformations in future projects  
    - To clean and prepare the test set  
    - To clean and prepare new data instances  
    - To make it easy to treat your preparation choices as hyperparameters  

1. Data cleaning:  
    - Fix or remove outliers (optional).  
    - Fill in missing values (e.g., with zero, mean, median...) or drop their rows (or columns).  
2. Feature selection (optional):  
    - Drop the attributes that provide no useful information for the task.  
3. Feature engineering, where appropriates:  
    - Discretize continuous features.  
    - Decompose features (e.g., categorical, date/time, etc.).  
    - Add promising transformations of features (e.g., log(x), sqrt(x), x^2, etc.).
    - Aggregate features into promising new features.  
4. Feature scaling: standardize or normalize features.  


In [ ]:
#remove snow_depth, horizontal visibility